## Load Modules

In [1]:
import torch
import pandas as pd
#from efficientnet_pytorch import EfficientNet
from datetime import date
import glob
from PIL import Image
from torchvision import  transforms,datasets, models
import torch.nn as nn

## Private variables

In [2]:
PATH_WEIGHTS = "/work/u2785625/AI_Cup_2022/Weights"
PUBLIC_TESTING_PATH = "/work/u2785625/AI_Cup_2022/ProcessedPublicTesting/"
folderNames = ['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f']
OUTPUT_CSV_PATH = "/work/u2785625/AI_Cup_2022/PublicTesting_OutputCsv/"

local_dir = "/work/u2785625/AI_Cup_2022/Datasets"
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Testing Transformation

In [3]:
test_tfm = transforms.Compose([ #valid and grading
    transforms.ToTensor(),
    transforms.Resize((456,456)),
    transforms.CenterCrop(456),
    transforms.Normalize([.5, 0.5, 0.5], [0.5, 0.5, 0.5]), 
])

## Load Model

In [4]:
model = models.efficientnet_b5(weights='DEFAULT').to(device)
num_ftrs = model.classifier[1].in_features
model.fc = nn.Linear(num_ftrs, n_classes)
model.load_state_dict(torch.load(PATH_WEIGHTS))
model = nn.DataParallel(model)

data = datasets.ImageFolder(root = local_dir, transform = test_tfm)

## Prediction

In [5]:

model.eval()
model.to(device)
List_FileName = list()
List_Output = list()ＦＦ
for folder in folderNames:
    folderPath = PUBLIC_TESTING_PATH + folder + '/'
    for fileName in glob.glob(folderPath + '*'):
        img = Image.open(fileName)
        x = test_tfm(img)
        x = x.unsqueeze(0)
        output = model(x)
        _, pred = torch.max(output, dim = 1)
        List_FileName.append(fileName[len(folderPath)::])
        List_Output.append(data.classes[pred])

In [6]:
output_dict = {"filename": List_FileName,
                "label": List_Output}
Out_dataframe = pd.DataFrame(output_dict)
print(Out_dataframe)

                                       filename           label
0      09462dbc-ecc4-45d7-8653-9e3a22bbacc2.jpg          longan
1      059b89e1-80c1-43a4-be36-cc911dc440f9.jpg      greenhouse
2      0ea762fd-124b-4db5-827a-60799a869870.jpg  chinesecabbage
3      0e03f42d-70b7-4671-aefe-3e7402ac90a9.jpg     cauliflower
4      0f8b5ffa-8609-4e05-afc0-56717bc0c113.jpg    bambooshoots
...                                         ...             ...
11143  f06bbcf8-f784-4810-92b9-c249c93f3f81.jpg            kale
11144  f2efc7a7-48a6-429a-be7f-86884826f223.jpg    bambooshoots
11145  fab7c542-6cc0-410c-8b33-8a9e0675029a.jpg      greenhouse
11146  fe0231e7-fdf1-4415-b586-e0e815b39ba4.jpg            kale
11147  fdc8142c-74ea-4cfe-baf8-bd9c70ed0da7.jpg     sweetpotato

[11148 rows x 2 columns]


## Save Csv

In [7]:
FileName =  date.today().strftime("%m/%d").replace('/','') + '.csv'

Out_dataframe.to_csv(OUTPUT_CSV_PATH + FileName , index = False)